<center>
    
# R406: Applied Economic Modelling with Python

</center>

<br> <br> 

<center>

## Numerical linear programming

</center>

<br><br> 

<center>
<b> Andrey Vassilev </b>
</center>

 

# Linear programming

The linear programming (LP) problem has the general form 
$$\min_{x} c \cdot x$$
s.t.
$$A_{ub} x \leq b_{ub}$$
$$A_{eq} x = b_{eq}$$
and, in some formulations, explicitly contains the additional constraints
$$lb \leq x \leq ub.$$

Here $c,x \in \mathbb{R}^n$, $A_{ub}\in \mathbb{R}^{m \times n}$, $b_{ub}\in \mathbb{R}^m$, $A_{eq}\in \mathbb{R}^{k \times n}$ and $b_{eq}\in \mathbb{R}^k$.

**Note:** There are different formulations of a LP problem. The one presented here is chosen to conform to the SciPy convention.

The `scipy.optimize` function that solves a LP problem is called `linprog`.

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.optimize import linprog, linear_sum_assignment

In simplified form, its syntax is 

```linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None)
```

Here `A_ub` and `A_eq` are 2D arrays, and `c`, `b_ub` and `b_eq` are 1D arrays.  
**Note:** Actually, they are array-like objects, which means that a list of lists should work for `A_ub` and `A_eq`. A safe choice would be to provide arrays directly.

Bounds can be supplied as $n \times 2$ array or a sequence of tuples.

Let us take the following LP problem:
$$\max_{x_1,x_2} x_1 + 2 x_2$$
s.t.  
$$x_1+x_2 \leq 5,$$
$$x_1\geq 0,$$
$$x_2\geq 0.$$

In [ ]:
Aub = np.array([[1,1]])
bub = np.array([5])
c = np.array([1,2]) 
sol = linprog(-c, A_ub=Aub, b_ub=bub) # The solver minimizes, 
                                      # hence the minus
sol

Consider a modified version of the previous problem, where the objective function takes the form $$ x_1 + C_1 x_2.$$

Try each of the following (a code cell with the necessary setup is available below) and interpret the results:
- Change `C1` to a number between 0 and 1, noninclusive. How does the solution change and why?
- Set `C1` equal to 1. 
   - What is the solution according to the solver? 
   - What's the value of the objective function? 
   - Use your NumPy skills to compute the value of the objective function for the vectors $x = (5,0)$, $x = (0,5)$ and $x = (2.5,2.5)$? Are these vectors feasible? What's going on?

- Change `C1` back to 2.0 and set a lower bound of 1 for $x_1$. What is the solution? What does the feasible set look like?
- Revert the lower bound for $x_1$ to zero and introduce upper bounds of 2.0 for both $x_1$ and $x_2$. What is the solution? What is the feasible set?
- Set the lower bound for $x_2$ equal to 10. What is the solution? What is the feasible set?

In [ ]:
Aub = np.array([[1,1]])
bub = np.array([5])
C1 = 2.0
c = np.array([1,C1])
bnd = np.array([[0,None],
                [0,None]])
sol = linprog(-c, A_ub = Aub, b_ub = bub, bounds = bnd)
sol

Let us take the modified LP problem:
$$\max_{x_1,x_2} x_1 + C_1 x_2$$
s.t.  
$$x_1+x_2 \leq 5,$$
$$1.5 x_1+x_2 \leq 6,$$
$$x_1\geq 0,$$
$$x_2\geq 0.$$

Try each of the following (a code cell with the necessary setup is available below) and interpret the results:
- What is the solution of the model for `C1` equal to 0.2, 1.0 and 5.0?
- Solve $$\min_{x_1,x_2} x_1 + C_1 x_2$$ s.t. the same constraints.
- For the preceding minimization problem, remove the lower bounds of 0 on the variables and solve it.

In [ ]:
Aub = np.array([[1.0,1.0],
                [1.5,1.0]])
bub = np.array([5,6])
C1 = 2.0
c = np.array([1,C1])
bnd = np.array([[0,None],
                [0,None]])
sol = linprog(-c, A_ub = Aub, b_ub = bub, bounds = bnd)
sol

# Some of the general principles of LP problems

- The feasible set (if nonempty) is a convex polyhedron.
- The solution (if there is one) is attained at a vertex or a convex combination of vertices.
- The level curves of the objective function are straight lines.

In [ ]:
# The LP problem objects
Aub = np.array([[1.0,1.0],
                [1.5,1.0]])
bub = np.array([5,6])
c = np.array([1,2])

# Auxiliary parameters
x1Lwr = -2 # Lower bound for plotting x1
x1Uppr = 8 # Upper bound for plotting x1
ObjLevelLwr = 1 # Lower bound for objective function value
                # used to define a set of level curves (lines)
ObjLevelUppr = 8 # Upper bound for objective function value
NrLevelLines = 5 # Number of level lines to be plotted

def ComputeLevelSet(c,l,npoints=100):
    X = np.linspace(l/c[0],0,npoints)
    Y = np.linspace(0,l/c[1],npoints)
    return X,Y
def ComputeConstraint(x1,A,b):
    """Solves the line equation A[0]*x1 + A[1]*x2 = b
       for the values of x2 given a range of values for x1"""
    return b/A[1]-A[0]/A[1]*x1
def ComputeVert(A,b):
    # The implementation below is pedestrian but hopefully transparent
    x0y0 = np.zeros(2) # the origin
    x0yn = np.array([0, min(b[0]/Aub[0,1],b[1]/Aub[1,1])])
    xny0 = np.array([min(b[0]/Aub[0,0],b[1]/Aub[1,0]),0])
    xnyn = sp.linalg.solve(A,b) # The intersection of the two lines
    stk = np.vstack((x0y0,x0yn,xnyn,xny0))
    return stk[:,0],stk[:,1] # x1 and x2 respectively

x1 = np.linspace(x1Lwr,x1Uppr)
yI1 = ComputeConstraint(x1,Aub[0,:],bub[0])
yI2 = ComputeConstraint(x1,Aub[1,:],bub[1])
Xv,Yv = ComputeVert(Aub,bub)

plt.plot(x1,yI1,'k',linewidth = 2)
plt.plot(x1,yI2,'k',linewidth = 2)
plt.axvline(x=0,color='k',linewidth = 2)
plt.axhline(y=0,color='k',linewidth = 2)
plt.fill(Xv,Yv,color = 'g', alpha = 0.5)
levels = np.linspace(ObjLevelLwr,ObjLevelUppr,NrLevelLines)
for l in levels:
    xO,yO = ComputeLevelSet(c,l,npoints = x1.size)
    plt.plot(xO,yO,'r',alpha=l/max(levels))
plt.xlabel(r'$x_1$',fontsize=16)
plt.ylabel(r'$x_2$',fontsize=16)
plt.show()

# Transportation problems

- There are $m$ origins that contain various amounts of a commodity that must be shipped to $n$ destinations to meet demand requirements.
- Origin $i$ supplies an amount $a_i \geq 0$, and destination $j$ demands amount $b_j \geq 0$.
- Shipping of the commodity from origin $i$ to destination $j$ incurs a cost $c_{ij} \geq 0$ per unit of commodity transported.
- The problem is to find the shipping pattern between origins and destinations that meets demands given available supply and minimizes the total shipping cost.
- A problem is called *balanced* if total supply equals total demand: $$\sum_{i=1}^m a_i = \sum_{j=1}^n b_j.$$

- Formally, the transportation problem is to find $x_{ij},~i=1,\ldots,m,~j=1,\ldots,n$ that minimize
$$ \sum_{i=1}^m \sum_{j=1}^n c_{ij}x_{ij}$$
subject to
$$\sum_{j=1}^n x_{ij} = a_i,~i=1,\ldots,m,$$
$$\sum_{i=1}^m x_{ij} = b_j,~j=1,\ldots,n,$$
$$x_{ij}\geq 0,~\forall i,j.$$
- **By virtue of its structure, a transportation problem always has an optimal solution.**

The equality constraints can be written more transparently as $$\begin{array}{llllllllllllll}
	x_{11} & +x_{12} & +\cdots & +x_{1n} &         &         &                     &         &         &         &         &         & =      & a_1 \\
	       &         &         &         & x_{21}  & +x_{22} & +\cdots             & +x_{2n} &         &         &         &         & =      & a_2 \\
	       &         &         &         &         &         &                     &         &         &         &         &         & \vdots &     \\
	       &         &         &         &         &         &                     &         & x_{m1}  & +x_{m2} & +\cdots & +x_{mn} & =      & a_m \\
	\hline
	x_{11} &         &         &         & +x_{21} &         & {\phantom{+}}\cdots &         & +x_{m1} &         &         &         & =      & b_1 \\
	       & x_{12}  &         &         &         & +x_{22} &                     &         &         & +x_{m2} &         &         & =      & b_2 \\
	       &         &         &         &         &         &                     &         &         &         &         &         & \vdots &     \\
	       &         &         & +x_{1n} &         &         &                     & +x_{2n} &         &         &         & +x_{mn} & =      & b_n
\end{array}$$

In matrix notation, the preceding takes the form 
$$A_{eq}x=b_{eq} \qquad \text{ with } \qquad A_{eq}=\left[\begin{array}{cccc}
\mathbf{1'} &  &  &  \\ 
 & \mathbf{1'} &  &  \\ 
 &  & \vdots &  \\ 
 &  &  & \mathbf{1'} \\ 
\mathbf{I} & \mathbf{I} & \cdots  & \mathbf{I} 
\end{array}\right],\quad x = \left[\begin{array}{c} x_{11}\\
x_{12} \\
\vdots\\
x_{1n}\\
x_{21}\\
\vdots\\
x_{mn}
\end{array}\right], \quad b_{eq} = \left[\begin{array}{c}a_1\\
\vdots\\
a_m\\
b_1\\
\vdots\\
b_n
\end{array}\right],$$
where $\mathbf{1}$ is an $n$-dimensional vector of ones and $\mathbf{I}$ is the $n\times n$ identity matrix.

Similarly, the objective function can equivalently be written as $$c \cdot x $$ where $x$ was defined above and $$c = \left[\begin{array}{c} c_{11}\\
c_{12} \\
\vdots\\
c_{1n}\\
c_{21}\\
\vdots\\
c_{mn}
\end{array}\right].$$


In [ ]:
def balanced_transportation_problem(a,b,C):
    import numpy as np
    import scipy as sp
    from scipy.optimize import linprog
    
    m = a.shape[0]
    n = b.shape[0]
    
    if np.any(a<0) or np.any(b<0) or np.any(C<0):
        raise Exception("Negative entries encountered.")
    if not a.shape[0] == C.shape[0]:
        raise Exception("Array shapes do not match.")
    if not b.shape[0] == C.shape[1]:
        raise Exception("Array shapes do not match.")
    if not np.sum(a) == np.sum(b):
        raise Exception("Problem is not balanced.")
     
    A_eq = np.vstack([np.array([np.roll(np.hstack([np.ones(n),np.zeros((m-1)*n)]),i) for i in range(0,m*n,n)]),
             np.tile(np.eye(n),m)])
    b_eq = np.hstack([a,b])
    c = C.flatten()
    bnd = (0,None)
    sol = linprog(c, A_eq = A_eq, b_eq = b_eq, bounds = bnd)
    if sol.status == 0:
        return (np.reshape(sol.x, C.shape),sol.message)
    else:
        return (None,sol.message)

In [ ]:
# Source: David G. Luenberger, Yinyu Ye. 2008. "Linear and Nonlinear Programming". 3rd ed. Springer. p. 176

a = np.array([25,25,50])
b = np.array([15, 20, 30, 35])
C = np.array([[10, 5, 6, 7],
              [8, 2, 7, 6],
              [9, 3, 4, 8]])

x,s = balanced_transportation_problem(a,b,C)    
print("Status:",s)
print("Solution:",x,sep='\n')

In [ ]:
# Source: David G. Luenberger, Yinyu Ye. 2008. "Linear and Nonlinear Programming". 3rd ed. Springer. p. 147

a = np.array([30, 80, 10, 60])
b = np.array([10, 50, 20, 80, 20])
C = np.array([[3, 4, 6, 8, 9],
              [2, 2, 4, 5, 5],
              [2, 2, 2, 3, 2],
              [3, 3, 2, 4, 2]])

x,s = balanced_transportation_problem(a,b,C)    
print("Status:",s)
print("Solution:",x,sep='\n')

# Linear sum assignment problems

- A special class of problems deals with how to optimally allocate $n$ objects to $n$ categories so as to minimize the total cost associated with each allocation choice. This can be viewed as a simplified version of a transportation problem.
- The classical example is assigning $n$ workers to $n$ jobs.
- Other examples:
  - Allocating taxis to customers depending on distance;
  - Assigning social security workers to districts;
  - Assigning teachers to classes.

## Formulation

- Formally, the linear sum assignment problem is to find $x_{ij},~i=1,\ldots,n,~j=1,\ldots,n$ that minimize
$$ \sum_{i=1}^n \sum_{j=1}^n c_{ij}x_{ij}$$
subject to
$$\sum_{j=1}^n x_{ij} = 1,~i=1,\ldots,n,$$
$$\sum_{i=1}^n x_{ij} = 1,~j=1,\ldots,n,$$
$$x_{ij}\geq 0,~\forall i,j.$$

- Some formulations explicitly require that $$x_{ij} \in \{0,1\},~\forall i,j.$$
- However, it can be shown that the solution will have this structure without imposing the requirement explicitly.

## Example 1

This is the [example](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html) from the Scipy docs. 

Let the cost matrix be $$\left [ \begin{matrix}
4 & 1 & 3 \\ 
2 & 0 & 5 \\ 
3 & 2 & 1
\end{matrix} \right ] .$$

Find the optimal assignment and the corresponding minimum cost.

In [ ]:
cost1 = np.array([[4, 1, 3], [2, 0, 5], [3, 2, 2]])
row_ind, col_ind = linear_sum_assignment(cost1)

print('Row indexes:',row_ind)
print('Column indexes:',col_ind)

Here is a more intuitive way to inspect the solution:

In [ ]:
S = np.zeros_like(cost1)
S[row_ind, col_ind] = 1
print('Solution:', S, sep = "\n")

This is how we can construct the optimal (minimum) cost:

In [ ]:
mincost = cost1[row_ind, col_ind].sum()
print(f'Minimum cost: {mincost:.1f}')

## Example 2

Source: https://www.slideshare.net/mobile/abubashars/assignment-problem-18034506

There are four workers who have to be assigned to four separate tasks. Each of them can complete any of the tasks but this will be associated with different costs. The costs are given by the following matrix, with rows corresponding to workers and columns – to tasks: 
$$\left [ \begin{matrix}
20 & 25 & 22 & 28\\ 
15 & 18 & 23 & 17 \\ 
19 & 17 & 21 & 24 \\ 
25 & 23 & 24 & 24
\end{matrix} \right ] .$$

Find an assignment that minimizes the total cost.

In [ ]:
cost2 = np.array([[20,25,22,28],
                 [15,18,23,17],
                 [19,17,21,24],
                 [25,23,24,24]])
row_ind, col_ind = linear_sum_assignment(cost2)

S = np.zeros_like(cost2)
S[row_ind, col_ind] = 1
print('Solution:', S, sep = "\n")

print(f'Minimum cost: {cost2[row_ind, col_ind].sum():.1f}') 

The solution is not necessarily unique:

In [ ]:
S = np.zeros_like(cost2)
S[row_ind, [0,3,1,2]] = 1
print('Candidate solution:', S, sep = "\n")

print(f'Associated cost: {cost2[row_ind, [0,3,1,2]].sum():.1f}')